In [25]:
import pandas as pd
import tensorflow_datasets as tfds
import numpy as np
import tqdm
import tensorflow as tf

In [26]:
df = pd.read_csv("nights_data.csv",index_col=0)
df = df[df["ds_type"]=="test"]

mask = df["night_name"].apply(lambda x:x=="Night")


In [27]:
def mapping(path,key):
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [260,260])
    return image, key

def copy_data(df,path_out):

    image_paths = "../" + df["Path"] + "/images/" + df["key"] + ".jpg"

    ds = tf.data.Dataset.from_tensor_slices((image_paths,df["key"]))
    ds = ds.map(mapping,num_parallel_calls=tf.data.AUTOTUNE)

    for img, path in tqdm.tqdm(tfds.as_numpy(ds)):
        
        tf.keras.utils.save_img(path_out + path.decode("utf-8") + ".jpg", img)

In [28]:
# copy_data(df[mask],f"Relabeled_Test_DS/Night/")
# copy_data(df[np.logical_not(mask)],f"Relabeled_Test_DS/Day/")

100%|██████████| 1608/1608 [00:06<00:00, 230.59it/s]


In [39]:
import os
data = dict()

for i in os.listdir("Relabeled_Test_DS/Night"):
    data[i[:-4]]="Night"

for i in os.listdir("Relabeled_Test_DS/Day"):
    data[i[:-4]]="Day"


df = pd.DataFrame(data.values(),index=data.keys(),columns=["Label"]).to_csv("Relabeled_Test_DS/cleaned_data.csv")